# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [ ]:
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA=LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [ ]:
Alu = lufact(A)

In [ ]:
typeof(Alu)

The different parts of the factorization can be extracted by special indexing

In [ ]:
Alu[:P]

In [ ]:
Alu[:L]

In [ ]:
Alu[:U]

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [ ]:
A\b

In [ ]:
Alu\b

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [ ]:
det(A) ≈ det(Alu)

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [ ]:
Aqr = qrfact(A)

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [ ]:
Aqr[:Q]

In [ ]:
Aqr[:R]

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [ ]:
Asym = A + A'
AsymEig = eigfact(Asym)

The values and the vectors can be extracted from the Eigen type by special indexing

In [ ]:
AsymEig[:values]

In [ ]:
AsymEig[:vectors]

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*Asym

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [ ]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [ ]:
Asym = A + A'
issymmetric(Asym)

but sometimes floating point error might get in the way.

In [ ]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

In [ ]:
issymmetric(Asym_noisy)

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [ ]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [ ]:
@time eigvals(Asym);

In [ ]:
@time eigvals(Asym_noisy);

In [ ]:
@time eigvals(Asym_explicit);

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [ ]:
1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [ ]:
Arational = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

In [ ]:
x = fill(1, (3))
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lufact(Arational)

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback